In [3]:
import numpy as np
from pathlib import Path
import shutil
import warnings

from eddymotion.data import pet
from eddymotion.model import PETModel
from eddymotion.viz import plot_dwi
import nibabel as nib
from eddymotion.data.splitting import lovo_split
from pathlib import Path

In [4]:
base_dir = Path("/Users/martinnorgaard/Documents/BIDS/")
bids_dir = base_dir / "ds004868-download"
derivatives_dir = base_dir / "eddymotion"

pet_file = bids_dir / "sub-01" / "ses-baseline" / "pet" / "sub-01_ses-baseline_pet.nii.gz"
json_file = bids_dir / "sub-01" / "ses-baseline" / "pet" / "sub-01_ses-baseline_pet.json"

pet_dataset = pet.PET.load(pet_file, json_file)

In [7]:
data_train, data_test = pet_dataset.lofo_split(15)

In [8]:
model = PETModel(timepoints=data_train[1], xlim = 7000)

In [9]:
model.fit(data_train[0])

In [6]:
predicted = model.predict(data_test[1])

In [7]:
# before
nifti_img_before = nib.Nifti1Image(predicted, pet_dataset.affine)
output_path_before = 'before_mc.nii'
nifti_img_before.to_filename(output_path_before)

# after
nifti_img_after = nib.Nifti1Image(data_test[0], pet_dataset.affine)
output_path_after = 'after_mc.nii'
nifti_img_after.to_filename(output_path_after)

In [5]:
from niworkflows.viz.notebook import display
import os
from IPython.display import SVG

moving_image = output_path_after
fixed_image = output_path_before
obj = display(fixed_image, moving_image, fixed_label='PET_before', moving_label='PET')

NameError: name 'output_path_after' is not defined

In [5]:
from eddymotion.estimator import PETMotionEstimator

In [6]:
estimator = PETMotionEstimator.estimate(pet_dataset, omp_nthreads=4, n_jobs=4)

Estimating PET motion:   0%|                                                                                                                                                                                                  | 0/27 [00:00<?, ?it/s]

240516-06:55:52,686 nipype.interface INFO:
	 stderr 2024-05-16T06:55:52.686793:WARNING: In /Users/martinnorgaard/Downloads/antsInstallExample-master/build/staging/include/ITK-5.2/itkCorrelationImageToImageMetricv4HelperThreader.hxx, line 85
240516-06:55:52,687 nipype.interface INFO:
	 stderr 2024-05-16T06:55:52.686793:CorrelationImageToImageMetricv4HelperThreader (0x600001870870): collected only zero points
240516-06:55:52,688 nipype.interface INFO:
	 stderr 2024-05-16T06:55:52.686793:
240516-06:55:52,704 nipype.interface INFO:
	 stderr 2024-05-16T06:55:52.704948:WARNING: In /Users/martinnorgaard/Downloads/antsInstallExample-master/build/staging/include/ITK-5.2/itkObjectToObjectMetric.hxx, line 579
240516-06:55:52,705 nipype.interface INFO:
	 stderr 2024-05-16T06:55:52.704948:CorrelationImageToImageMetricv4 (0x13de05b60): No valid points were found during metric evaluation. For image metrics, verify that the images overlap appropriately. For instance, you can align the image centers by

Estimating PET motion: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 27/27 [04:13<00:00,  9.40s/it]


In [7]:
estimator

[array([[ 9.18308029e-01,  3.14120418e-03,  9.43561966e-03,
          1.47570478e+01],
        [-1.91521608e-03,  9.99754542e-01, -6.47607787e-04,
         -1.17171090e+00],
        [-2.63604311e-01,  1.74485385e-02,  1.05247613e+00,
          8.23569593e+01],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00]]),
 array([[ 9.90934729e-01,  8.92484657e-04,  2.80051646e-03,
          4.37107351e+00],
        [ 4.90719283e-04,  9.99946349e-01, -1.51047448e-04,
         -2.54520226e-01],
        [-5.70326698e-03,  3.12829167e-05,  1.00007144e+00,
          1.01836908e-01],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00]]),
 array([[ 1.00478330e+00,  1.01585891e-05,  1.13635091e-04,
          2.18816249e-02],
        [-1.27204231e-03,  9.99909317e-01, -3.51268600e-04,
         -4.41583103e-01],
        [-6.51641816e-04,  6.10237348e-06,  1.00026065e+00,
          6.16551122e-02],
        [ 0.00000000e+00,  0.00000000e+

In [8]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

# For IPython or Python script, you might use
import matplotlib
matplotlib.use('TkAgg')

def extract_motion_parameters(affine):
    # Translation is straightforward
    tx, ty, tz = affine[0, 3], affine[1, 3], affine[2, 3]
    
    # For small rotations, these approximations are usually sufficient
    rx = np.arctan2(affine[2, 1], affine[2, 2])
    ry = np.arctan2(-affine[2, 0], np.sqrt(affine[2, 1]**2 + affine[2, 2]**2))
    rz = np.arctan2(affine[1, 0], affine[0, 0])
    
    return tx, ty, tz, np.degrees(rx), np.degrees(ry), np.degrees(rz)

In [9]:
# Assuming 'affines' is the list of affine matrices you have
motion_parameters = []
affines = estimator

for idx, affine in enumerate(affines):
    tx, ty, tz, rx, ry, rz = extract_motion_parameters(affine)
    motion_parameters.append({
        "Frame": idx,
        "Translation X": tx,
        "Translation Y": ty,
        "Translation Z": tz,
        "Rotation X (deg)": rx,
        "Rotation Y (deg)": ry,
        "Rotation Z (deg)": rz
    })

df_motion = pd.DataFrame(motion_parameters)

In [ ]:
# Set up the matplotlib figure
plt.figure(figsize=(18, 10))

# Translation plots
plt.subplot(2, 3, 1)
sns.lineplot(x='Frame', y='Translation X', data=df_motion)
plt.title('Translation X over Frames')

plt.subplot(2, 3, 2)
sns.lineplot(x='Frame', y='Translation Y', data=df_motion)
plt.title('Translation Y over Frames')

plt.subplot(2, 3, 3)
sns.lineplot(x='Frame', y='Translation Z', data=df_motion)
plt.title('Translation Z over Frames')

# Rotation plots
plt.subplot(2, 3, 4)
sns.lineplot(x='Frame', y='Rotation X (deg)', data=df_motion)
plt.title('Rotation X over Frames')

plt.subplot(2, 3, 5)
sns.lineplot(x='Frame', y='Rotation Y (deg)', data=df_motion)
plt.title('Rotation Y over Frames')

plt.subplot(2, 3, 6)
sns.lineplot(x='Frame', y='Rotation Z (deg)', data=df_motion)
plt.title('Rotation Z over Frames')

plt.tight_layout()
plt.show()